# Méthode d'analyse


# - 1) imports

In [43]:
sheetname = None

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [2]:
from jr_data_science import functions_de_decouverte_de_fichiers
from jr_data_science import useful_functions

In [3]:
def format_euro(nombre, devise=u"€"):
    nombre = format_nombre(nombre)
    return " ".join([nombre, devise])


def format_nombre(nombre):
    import numpy as np
    if np.isnan(nombre):
        return "0"

    nombre = float(nombre)
    if nombre < 1e3:
        return "{:.1f}".format(nombre)
    if nombre < 1e6:
        return "{:.1f}K".format(nombre / 1e3)
    if nombre < 1e9:
        return "{:.1f}M".format(nombre / 1e6)
    if nombre < 1e12:
        return "{:.1f}G".format(nombre / 1e9)
    return str(nombre)


In [4]:
def print_list_as_dict(liste):
    print("dico = {")
    for key in liste:
        print("\"{key}\" : str ,".format(key=key))
    print("}")
    

In [5]:
def print_col_as_dict(df):
    """
    Générateur de code : affiche à l'écran la liste des colonnes d'une DF sous forme de dico.
    """
    print("dtypes = {")
    for col in df.columns:
        print("\"{col}\" : str ,".format(col=col))
    print("}")
    print("""
# => changez les types dans les colonnes, et mettez les dates et colonnes de catégories dans les tableaux ci dessous")
dayfirst = -1
parse_dates = []
categories = []
# rechargez la df avec les colonnes indiquées
df = pd.read
df = pd.read_csv(path, sep=sep, dtype=dtypes, parse_dates=parse_dates, dayfrist=dayfirst)
df.shape, df.dtypes
# vous pouvez aussi générer du code ensuite pour les colonnes catégorielles
for col in categories:
code_for_cat_col(df, col)    
    """)


In [162]:
def analyse_to_excel(df, filename, output_dir="."):
    """
    Construit un fichier csv avec une analyse de chaque colonne de la df => à importer dans excel en tant que tsv.

    """
    import os
    output_file = os.path.join(output_dir, "%s_analyse.csv"%filename)
    
    fichier = [" "]*15
    nz      = 0
    n_col   = -1
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("%s lignes, %s col"%df.shape)
    for col in df.columns:
            print(col)
            n_col        += 1
            nb_valeur     = df[col].value_counts().shape[0]
            nb_null       = df[col].isnull().sum()
            n             = 10
            top_n         = df[col].value_counts().sort_values(ascending=False).reset_index()[:n].values
            pct_top_n     = [x[1]/df.shape[0] for x in top_n]
            fichier[0]   += "\t%s\t  _ \t  _ "%(col)
            fichier[1]   += "\tnb_valeur \t%s \t _"%nb_valeur
            fichier[2]   += "\tnb_null \t %s \t _"%(nb_null)
            current_ligne = 2
            for ((x,y),z) in zip(top_n, pct_top_n):
                current_ligne += 1
                fichier[current_ligne] += "\t%s\t %s \t %.2f"%(x,y,z)
            while current_ligne<14:
                current_ligne += 1
                fichier[current_ligne] += "\t%s\t %s \t %s"%("_","_","_")
            if n_col>0:
                if n_col %4==0:        
                    print(n_col)
                    with open(output_file, "a", encoding="utf-8") as f:
                        for line in fichier:
                            f.write(line+"\n")
                    fichier = [" "]*15
    with open(output_file, "a", encoding="utf-8") as f:
        for line in fichier:
            f.write(line+"\n")
            
    print("Wrote into %s"%(output_file))

In [163]:
def code_for_cat_col(df, col):
    """
    Generate a python string to help transforming a pandas column into a categorie.
    """
    print("from pandas.api.types import CategoricalDtype")
    to_print = "cat = CategoricalDtype(categories=["
    for x in sorted(df[col].unique()):
        to_print += "\'%s\',"%x
    to_print += "], ordered=True)"
    print(to_print)
    print("#df['%s'] = df['%s'].astype(cat)"%(col, col))

In [164]:
def plot_pivot(graph):
    tab = graph["df"].pivot_table(**graph["pivot"])
    sns.heatmap(tab.sort_index(), cmap="PiYG", center=0)
    plt.title(graph["title"]);

In [165]:
def print_cols(liste):
    for i,x in enumerate(liste):
        print("%s : %s"%(i,x))

# 0) chargement des données

In [350]:

data_dir = "/Users/romain/Google Drive/ayming/BAT_DATAS"

In [351]:
file_extension = ".csv"
file_extension = ".xlsx"
csvs = functions_de_decouverte_de_fichiers.get_all_specific_paths_under_dir(data_dir, file_extension)
csvs = sorted([x for x in csvs if "/~" not in x ])

In [352]:
print_cols(csvs)

0 : /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_COLLECTE.xlsx
1 : /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_COLLECTE_analyse.xlsx
2 : /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DELAIS_TX.xlsx
3 : /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DELAIS_TX_analyse.xlsx
4 : /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DOSSIERS.xlsx


In [276]:
path = csvs[2]  
path

'/Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DELAIS_TX.xlsx'

In [277]:

if file_extension.endswith("xlsx"):
    sheetname = input("Quelle feuille ?")
    print("Feuille qui sera analysée : ", sheetname)

Quelle feuille ?BAT_DELAIS_TX
Feuille qui sera analysée :  BAT_DELAIS_TX


# 1) Charger la df, compter les lignes et colonnes

In [332]:
if path.endswith("csv"):
    print("going to read a csv")
    df = pd.read_csv(path, functions_de_decouverte_de_fichiers.find_delimiter(path))
if path.endswith("xlsx"):
    print("going to read an excel file")
    print(path)
    if not sheetname:
        sheetname = input("Which sheetname")
    print(sheetname)
    df = pd.read_excel(path, sheetname=sheetname)

going to read an excel file
/Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DELAIS_TX.xlsx
BAT_DELAIS_TX


/Users/romain/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


## 1.1) taille des données : 

In [333]:
df.shape

(29999, 14)

## 1.2) nombre de lignes pleines : 

In [334]:
df.dropna().shape

(13351, 14)

# 1.3) nombre de nuls par ligne

In [335]:
to_print = "{col: <15} : {nb_null}"
for i, col in enumerate(sorted(df.columns, key=lambda x:sum(df[x].isnull()), reverse=True)):
    print(i, to_print.format(col=col, nb_null=sum(df[col].isnull())))


0 DATE SAISINE    : 16253
1 ETAB_ITN        : 16053
2 ORGANISME       : 15888
3 DEMARCHE        : 15767
4 DATE DECISION   : 15767
5 RESULTAT        : 15767
6 ANNEE           : 15767
7 MOIS            : 15767
8 N° EVT          : 15767
9 EVT             : 15767
10 RAISON SOCIALE  : 15767
11 CONSULTANT      : 15767
12 EQUIPE          : 15767
13 DELAI           : 0


In [336]:
df.sample(2).T

,10785,19409
DEMARCHE,TCI,NaN
DATE DECISION,2018-07-13 00:00:00,NaT
DATE SAISINE,2016-07-11 00:00:00,NaT
ORGANISME,TCI DE MARSEILLE,NaN
RESULTAT,Positif,NaN
ANNEE,2018,NaN
MOIS,7,NaN
N° EVT,8.48215e+06,NaN
EVT,SLIMANI,NaN
ETAB_ITN,386639,NaN


In [337]:
df.tail()

,DEMARCHE,DATE DECISION,DATE SAISINE,ORGANISME,RESULTAT,ANNEE,MOIS,N° EVT,EVT,ETAB_ITN,RAISON SOCIALE,CONSULTANT,EQUIPE,DELAI
29994,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non calculable
29995,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non calculable
29996,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non calculable
29997,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non calculable
29998,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non calculable


# 2) on crée un fichier excel qui extrait les top 10 de chaque colonne pour avoir une vue d'ensemble

In [338]:
def get_filename(path):
    import os
    return os.path.basename(path).split(".")[0]

In [339]:
import os
analyse_to_excel(df, get_filename(path), os.path.dirname(path))

DEMARCHE
DATE DECISION
DATE SAISINE
ORGANISME
RESULTAT
4
ANNEE
MOIS
N° EVT
EVT
8
ETAB_ITN
RAISON SOCIALE
CONSULTANT
EQUIPE
12
DELAI
Wrote into /Users/romain/Google Drive/ayming/BAT_DATAS/BAT_DATAS_DELAIS_TX_analyse.csv


# 3) En lisant l'excel on peut définir les types de chaque colonne

In [340]:
index_de_test = df.dropna().sample(2).index
df.ix[index_de_test].T

/Users/romain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,276,4556
DEMARCHE,CRA,TASS
DATE DECISION,2017-03-27 00:00:00,2016-09-21 00:00:00
DATE SAISINE,2016-12-16 00:00:00,2012-09-14 00:00:00
ORGANISME,MOSELLE,TASS DU RHONE
RESULTAT,Négatif,Positif
ANNEE,2017,2016
MOIS,3,9
N° EVT,1.47376e+07,7.14533e+06
EVT,JACOB HOLTZ,BELGOUR
ETAB_ITN,383328,36180


In [341]:
tmp_df = "temp_df_to_re_read.csv"
df.fillna("", inplace=True)
df.to_csv(tmp_df, index=False)

In [342]:
print_col_as_dict(df)

dtypes = {
"DEMARCHE" : str ,
"DATE DECISION" : str ,
"DATE SAISINE" : str ,
"ORGANISME" : str ,
"RESULTAT" : str ,
"ANNEE" : str ,
"MOIS" : str ,
"N° EVT" : str ,
"EVT" : str ,
"ETAB_ITN" : str ,
"RAISON SOCIALE" : str ,
"CONSULTANT" : str ,
"EQUIPE" : str ,
"DELAI" : str ,
}

# => changez les types dans les colonnes, et mettez les dates et colonnes de catégories dans les tableaux ci dessous")
dayfirst = -1
parse_dates = []
categories = []
# rechargez la df avec les colonnes indiquées
df = pd.read
df = pd.read_csv(path, sep=sep, dtype=dtypes, parse_dates=parse_dates, dayfrist=dayfirst)
df.shape, df.dtypes
# vous pouvez aussi générer du code ensuite pour les colonnes catégorielles
for col in categories:
code_for_cat_col(df, col)    
    


In [343]:
sep

','

In [344]:
df = pd.read_csv(tmp_df)
df.dtypes

DEMARCHE           object
DATE DECISION      object
DATE SAISINE       object
ORGANISME          object
RESULTAT           object
ANNEE             float64
MOIS              float64
N° EVT            float64
EVT                object
ETAB_ITN          float64
RAISON SOCIALE     object
CONSULTANT         object
EQUIPE             object
DELAI              object
dtype: object

In [349]:
df.DELAI.replace("non calculable", -1, inplace=True)

0          35
1          77
2          55
3         471
4        1217
5         710
6         185
7        1596
8         507
9         702
10        605
11        679
12        713
13        123
14        103
15        269
16        194
17        898
18         82
19        688
20       1307
21       1523
22        367
23       1175
24        602
25        415
26        124
27          6
28        567
29         91
         ... 
29969      -1
29970      -1
29971      -1
29972      -1
29973      -1
29974      -1
29975      -1
29976      -1
29977      -1
29978      -1
29979      -1
29980      -1
29981      -1
29982      -1
29983      -1
29984      -1
29985      -1
29986      -1
29987      -1
29988      -1
29989      -1
29990      -1
29991      -1
29992      -1
29993      -1
29994      -1
29995      -1
29996      -1
29997      -1
29998      -1
Name: DELAI, Length: 29999, dtype: object

In [347]:
dtypes = {
"DEMARCHE" : str ,
"DATE DECISION" : str ,
"DATE SAISINE" : str ,
"ORGANISME" : str ,
"RESULTAT" : str ,
"ANNEE" : float ,
"MOIS" : float ,
"N° EVT" : str ,
"EVT" : str ,
"ETAB_ITN" : str ,
"RAISON SOCIALE" : str ,
"CONSULTANT" : str ,
"EQUIPE" : str ,
"DELAI" : int ,
}

# => changez les types dans les colonnes, et mettez les dates et colonnes de catégories dans les tableaux ci dessous")
dayfirst = -1
parse_dates = ["DATE DECISION", "DATE SAISINE", ]
parse_dates = [1, 2]
categories = ["RESULTAT"]
# rechargez la df avec les colonnes indiquées
df = pd.read_csv(tmp_df, sep=sep, dtype=dtypes, parse_dates=parse_dates, dayfirst=dayfirst)
df.shape, df.dtypes
# vous pouvez aussi générer du code ensuite pour les colonnes catégorielles
for col in categories:
    code_for_cat_col(df, col)

ValueError: invalid literal for int() with base 10: 'non calculable'

In [238]:
df.dtypes

Unnamed: 0              int64
EQUIPE                 object
CONSULTANT             object
N° DEP                 object
TYPE DEP               object
n° EVT                 object
EVT                    object
DATE EVT       datetime64[ns]
TYPE EVT               object
COMMENT EVT            object
CE                     object
TX IPP                float64
NB IJ                 float64
ETAB_RS                object
ETAB_ITN               object
ETAB_VILLE             object
ETAB_SIRET             object
ETAB_GRP               object
DT_CREATION    datetime64[ns]
ANNEE CREA            float64
MOIS CREA             float64
TRANCHE                object
dtype: object

# On peut vérifier la cohérence entre les nouvelles data formatées et le print ci dessus 

In [239]:
df.ix[index_de_test].T

/Users/romain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,9894,19313
Unnamed: 0,9894,19313
EQUIPE,KRAWCZYK,BEN TAHAR
CONSULTANT,MICHAUT,SZUMNY
N° DEP,15426935.0,16085799.0
TYPE DEP,1.0,1.0
n° EVT,15426933.0,16085797.0
EVT,LOYER,BELARBI
DATE EVT,2016-09-20 00:00:00,2016-04-15 00:00:00
TYPE EVT,MP30,AT
COMMENT EVT,PAS D'enjeux pour le moment car pas d'arrêt sa...,rechute du 11/08/17 mais conso le 08/07/17


# 4) gestion des colonnes catégorielles

In [240]:
categories = ["TRANCHE"]

In [241]:
from pandas.api.types import CategoricalDtype
for col in categories:
    code_for_cat_col(df, col)
# Copiez le code ci dessous dans des cellules pour les exécuter

from pandas.api.types import CategoricalDtype
cat = CategoricalDtype(categories=['AUTRES','IP INDEFINI','IP1','IP2','IP3','IP4','IT INDEFINI','IT1','IT2','IT3','IT4','IT5','IT6',], ordered=True)
#df['TRANCHE'] = df['TRANCHE'].astype(cat)


In [242]:
cat = CategoricalDtype(categories=['IP1','IP2','IP3','IP4','IP INDEFINI','IT INDEFINI','IT1','IT2','IT3','IT4','IT5','IT6','AUTRES',], ordered=True)
df['TRANCHE'] = df['TRANCHE'].astype(cat)

In [243]:
df.dtypes

Unnamed: 0              int64
EQUIPE                 object
CONSULTANT             object
N° DEP                 object
TYPE DEP               object
n° EVT                 object
EVT                    object
DATE EVT       datetime64[ns]
TYPE EVT               object
COMMENT EVT            object
CE                     object
TX IPP                float64
NB IJ                 float64
ETAB_RS                object
ETAB_ITN               object
ETAB_VILLE             object
ETAB_SIRET             object
ETAB_GRP               object
DT_CREATION    datetime64[ns]
ANNEE CREA            float64
MOIS CREA             float64
TRANCHE              category
dtype: object

# 7) on crée des colonnes de jours / semaines sur les dates

In [244]:
def decompose_date(df, analyse_col):
    print(analyse_col)
    import pandas as pd
    clean_col = analyse_col.replace(" ", "_")
    from collections import OrderedDict
    weekdays = OrderedDict(
        {0: "lundi", 1: "mardi", 2: "mercredi", 3: "jeudi", 4: "vendredi", 5: "samedi", 6: "dimanche"})
    mois = OrderedDict({1: "janvier", 2: "février", 3: "mars", 4: "avril", 5: "mai", 6: "juin", 7: "juillet", 8: "août",
                        9: "septembre", 10: "octobre", 11: "novembre", 12: "décembre"})
    #
    new_col = "%s_jour_semaine" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.weekday()).map(weekdays)
    df[new_col] = pd.Categorical(df[new_col], categories=weekdays.values(), ordered=True)
    #
    new_col = "%s_jour_semaine_int" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.weekday())

    #
    new_col = "%s_jour_mois" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.day)
    df[new_col] = pd.Categorical(df[new_col], categories=range(32), ordered=True)

    #
    new_col = "%s_mois" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.month).map(mois)
    df[new_col] = pd.Categorical(df[new_col], categories=mois.values(), ordered=True)
    #
    new_col = "%s_nth_mois" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.month)
    df[new_col] = pd.Categorical(df[new_col], categories=range(1, 13), ordered=True)
    #
    new_col = "%s_week" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: int(x.week))
    df[new_col] = pd.Categorical(df[new_col], categories=range(53), ordered=True)

    #
    new_col = "%s_year" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: int(x.year))
    df[new_col] = pd.Categorical(df[new_col], categories=sorted(list(df[new_col].unique())), ordered=True)
    #
    new_col = "%s_nth_day" % clean_col
    df[new_col] = df[analyse_col].map(lambda x: x.timetuple().tm_yday)
    df[new_col] = pd.Categorical(df[new_col], categories=range(366), ordered=True)
    print("fini")


In [245]:
from collections import OrderedDict

In [246]:
weekdays = OrderedDict({0: "lundi", 1:"mardi", 2:"mercredi", 3:"jeudi", 4:"vendredi", 5:"samedi", 6:"dimanche"})
mois     = OrderedDict({1:"janvier", 2:"février", 3:"mars", 4:"avril", 5: "mai", 6:"juin", 7:"juillet", 8:"août", 9:"septembre", 10:"octobre", 11:"novembre", 12:"décembre"})

In [247]:
#parse_dates = {"Created on" : "%d.%m.%Y", 
#               "Follow Up Date" : "%d.%m.%Y", 
#               "Invoice expect. date (QMT)" : "%d.%m.%Y"}
parse_dates


['DATE EVT', 'DT_CREATION']

In [248]:
df.dtypes

Unnamed: 0              int64
EQUIPE                 object
CONSULTANT             object
N° DEP                 object
TYPE DEP               object
n° EVT                 object
EVT                    object
DATE EVT       datetime64[ns]
TYPE EVT               object
COMMENT EVT            object
CE                     object
TX IPP                float64
NB IJ                 float64
ETAB_RS                object
ETAB_ITN               object
ETAB_VILLE             object
ETAB_SIRET             object
ETAB_GRP               object
DT_CREATION    datetime64[ns]
ANNEE CREA            float64
MOIS CREA             float64
TRANCHE              category
dtype: object

In [250]:
for col_date in parse_dates:
    df[col_date] = df[col_date].fillna(datetime.date(1970,1,1))
    decompose_date(df, col_date)
df.dtypes

DATE EVT
fini
DT_CREATION
fini


Unnamed: 0                               int64
EQUIPE                                  object
CONSULTANT                              object
N° DEP                                  object
TYPE DEP                                object
n° EVT                                  object
EVT                                     object
DATE EVT                        datetime64[ns]
TYPE EVT                                object
COMMENT EVT                             object
CE                                      object
TX IPP                                 float64
NB IJ                                  float64
ETAB_RS                                 object
ETAB_ITN                                object
ETAB_VILLE                              object
ETAB_SIRET                              object
ETAB_GRP                                object
DT_CREATION                     datetime64[ns]
ANNEE CREA                             float64
MOIS CREA                              float64
TRANCHE      

# on split les valeurs en eur pour récupérer les montants

In [252]:
def fetch_devise(string):
    """
    cherche les mots ["EUR", "$", "£"] dans la string et le renvoie si trouvé.
    """
    devises = ["EUR", "$", "£"]
    default = "unknown"
    for devise in devises:
        if devise in string:
            return devise
    return default
def fetch_value(string):
    """
    Cherche les nombres dans la string.
    """
    import re
    regex = re.compile("\d+(?:\.\d+)*")
    return regex.findall(string)    
        

In [253]:
#df["devise"] = df['Expected Sales Vol.'].map(fetch_devise)

In [254]:
import re
regex = re.compile("\d+(?:\.\d+)*")

In [255]:
#df["montant"] = df['Expected Sales Vol.'].map(regex.findall).map(lambda x: int(x[0].replace(".", "")))

# 6) Ici on doit avoir une df propre

In [257]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,49989,49990,49991,49992,49993,49994,49995,49996,49997,49998
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,49989,49990,49991,49992,49993,49994,49995,49996,49997,49998
EQUIPE,TOU,MAUREL,CARRILLO,TOU,BEN TAHAR,CARRILLO,REFFAY,DEVIN,CERNAY,BEN TAHAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CONSULTANT,HOULLE,EYEBE EYEBE,JANNING,HOULLE,DOMPERT,ZERROU,BUIZZA,COUTURIER,GUILLAUME,LESPINET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N° DEP,12198796.0,13145614.0,13297342.0,13406351.0,13408421.0,13410004.0,13417552.0,13423791.0,13424749.0,13429238.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TYPE DEP,1.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
n° EVT,9993501.0,12869426.0,13297340.0,11152687.0,13256242.0,13410002.0,13417543.0,12303981.0,13424743.0,13429232.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EVT,LUCAS,CHAIAHELOUDJOU,SOUFANE,FONTELLES,DANG,SICRE,DAVID,DA FONSECA,JOUANNEST,WOZNIAK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATE EVT,2009-07-01 00:00:00,2014-10-28 00:00:00,2016-06-25 00:00:00,2013-03-16 00:00:00,2014-06-19 00:00:00,1970-01-01 00:00:00,2014-10-23 00:00:00,2014-03-22 00:00:00,2014-11-25 00:00:00,2015-05-21 00:00:00,...,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00
TYPE EVT,MP,MP65,AT,MP,MP57,AT,AT,MP57,AT,MP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMMENT EVT,NaN,Pas de motif de compte spé,NaN,NaN,MP n° 140619446 : rupture de la coiffe des rot...,NaN,établissement a priori fermé - attendre confir...,NaN,NaN,Procédure FI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
